In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

import re

import json

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get(input_.loc[a, 'Url'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                title = html.xpath('//h1[@itemprop="name"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                part_number = html.xpath('//dd[@itemprop="sku"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//div[@class="productView-img-container"]/a/@href')[0].strip()

                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(html.xpath('//div[@id="addon"]/@data-custom-fields')[0])

                # = = = = = = = = = = = = = = =
                
                theme_bundle = re.search('window.__webpack_public_path__ = ".*?dist/"', resp).group().split('"')[1] + 'theme-bundle.main.js'

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Part_Number': part_number,
                                         'Title': title,
                                         'Application_Summary': '',
                                         'Vehicle_1': '',
                                         'Vehicle_2': '',
                                         'OE': '',
                                         'Pic': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Src': src,
                                         'Theme_Bundle': theme_bundle}])

                for data in json_data:
                    if data['name'].strip() == 'Application_Summary':
                        df_temp.loc[0, 'Application_Summary'] = '\n'.join([vehicle.strip() for vehicle in data['value'].strip().split(';')])
                    else:
                        df_temp.loc[0, data['name'].strip().lower()] = data['value'].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：96

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-10019
[尝试次数：1] - [剩余数量：76] - [当前时间：17:22:50]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-10024
[尝试次数：1] - [剩余数量：75] - [当前时间：17:22:50]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-10020
[尝试次数：1] - [剩余数量：74] - [当前时间：17:22:51]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-10006
[尝试次数：1] - [剩余数量：73] - [当前时间：17:22:51]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-10014
[尝试次数：1] - [剩余数量：72] - [当前时间：17:22:51]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-10021
[尝试次数：1] - [剩余数量：71] - [当前时间：17:22:51]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-10022
[尝试次数：1] - [剩余数量：70] - [当前时间：17:22:51]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-10005
[尝试次数：1] - [剩余数量：69] - [当前时间：17:22:51]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-1000
[尝试次数：1] - [剩余数量：68] - [当前时间：17:22

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-1075l
[尝试次数：1] - [剩余数量：1] - [当前时间：17:23:03]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-1051
[尝试次数：1] - [剩余数量：0] - [当前时间：17:23:03]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-1060
[尝试次数：1] - [剩余数量：0] - [当前时间：17:23:03]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-1049
[尝试次数：1] - [剩余数量：0] - [当前时间：17:23:03]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-1077
[尝试次数：1] - [剩余数量：0] - [当前时间：17:23:03]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-1053
[尝试次数：1] - [剩余数量：0] - [当前时间：17:23:03]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-1080
[尝试次数：1] - [剩余数量：0] - [当前时间：17:23:03]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-1062
[尝试次数：1] - [剩余数量：0] - [当前时间：17:23:04]

[ok] - https://www.cardone.com/product/windshield-wiper-motor-40-1084
[尝试次数：1] - [剩余数量：0] - [当前时间：17:23:04]

[ok] - https://www